Constance Delannoy

APPM 4650 | Homework 4

July 8, 2020

In [11]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import math

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="The raw code is by default hidden for easier reading.To toggle on/off the raw code, click here."></form>''')

__Question 1__

We have $\int_0^1 f(x)dx = af(\frac{1}{3}) + bf(\frac{2}{3})$

Let $t = 2x-1$ so that: \begin{cases} x=0 \rightarrow t = -1 \\ = x=1 \rightarrow t-1 \\ x = (t+1)/2\\ \frac{dt}{dx} = 2\end{cases}

The problem becomes: $\int_0^1 f(x)dx = \int_{-1}^1 2d(\frac{t+1}{2})dt$.

Using the direct method, we now get:

$\int_{-1}^1 2dt = 2x \rvert_{-1}^1 = 4 \rightarrow a+b = 4$

$\int_{-1}^1 2xdt = x^2 \rvert_{-1}^1 = 0 \rightarrow \frac{a}{3} + \frac{2b}{3} = 0 \rightarrow a = -2b$

Thus: $a=8$ and $b=-4$ so that:

$\int_0^1 f(x)dx = 8f(\frac{1}{3}) - 4f(\frac{2}{3})$

__Question 2__

In [27]:
def rk4(f1, f2, f1true, f2true, a, b, h):
    "applying RK-4 to system of coupled ODEs when we know the truth"
    
    res = {'x':[], 'y1_est':[], 'y2_est': [], 'y1_true': [], 'y2_true':[]}
    steps = list(map(lambda x: x/10, range(0, 11, 2)))
    
    for t in steps:
        k11 = h*f1(t, a, b)
        k12 = h*f2(t, a, b)
        
        k21 = h*f1(t + h/2, a + k11/2, b + k12/2)
        k22 = h*f2(t + h/2, a + k11/2, b + k12/2)
        
        k31 = h*f1(t + h/2, a + k21/2, b + k22/2)
        k32 = h*f2(t + h/2, a + k21/2, b + k22/2)
        
        k41 = h*f1(t + h/2, a + k31/2, b + k32/2)
        k42 = h*f2(t + h/2, a + k31/2, b + k32/2)
        
        f1_est = a + (1/6)*(k11 + 2*k21 + 2*k31 + k41)
        f2_est = b + (1/6)*(k21 + 2*k22 + 2*k32 + k42)
        
        f1_true = f1true(t)
        f2_true = f2true(t)
        
        res['x'].append(t)
        res['y1_est'].append(f1_est)
        res['y2_est'].append(f2_est)
        res['y1_true'].append(f1_true)
        res['y2_true'].append(f2_true)
        
        a = f1_est
        b = f2_est
        
    return res

In [28]:
def fun1(t, a, b):
    "this is u'1 in the homework"

    y = 3*a + 2*b - (2*t**2 + 1)*math.exp(2*t)
    
    return y

def fun2(t, a, b):
    "this is u'2 in the homework"
    
    y = 4*a + b + (t**2 + 2*t - 4)*math.exp(2*t)
    
    return y

def fun1_true(t):
    "this is u1(t) in the homework"
    
    y = (1/3)*math.exp(5*t) - (1/3)*math.exp(-t) + math.exp(2*t)
    
    return y

def fun2_true(t):
    "this is u2(t) in the homework"
    
    y = (1/3)*math.exp(5*t) + (2/3)*math.exp(-t) + (t**2)*math.exp(2*t)
    
    return y

In [31]:
pd.DataFrame(rk4(fun1, fun2, fun1_true, fun2_true, 0, 1, 0.2))

,x,y1_est,y1_true,y2_est,y2_true
0,0.0,0.077567,1.000000,0.395330,1.000000
1,0.2,-0.403175,2.125008,-0.774941,1.511587
2,0.4,-2.577905,4.465120,-3.559897,3.265985
3,0.6,-9.543849,9.832359,-10.875353,8.256295
4,0.8,-29.565454,23.002639,-30.697015,21.668877
5,1.0,-84.429249,56.737483,-84.524080,57.105362


__Question 3__